In [1]:
import psycopg2
import configparser

config = configparser.ConfigParser()
config.read("../../config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

def execute_sql(SQL):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

# Calculating accessibility statistics based on 2FSCA method
First, we create the table

## Step 1 table and calculations

In [15]:
SQL = """

DROP TABLE IF EXISTS step1_stats;

CREATE TABLE IF NOT EXISTS step1_stats (
    --- There is no need to store POI category in step1 statistics
    --- Any POI with the same originH3 will have the same step 1 value!
    --- If we were to store it, we can't easily reuse step1 values for adding new POIs of a new type in H3id
	--- The same applies for groupname - we only need to store TOTAL population per catchment at this stage
    --- We keep categorytype as some populations == people and some == households
	timeofday varchar(50),
	categorytype text,	
	catchmentid bigint,
    h3id character(15),
	ratio float
);

WITH step1 AS (
         SELECT 
                c3m.h3id,                
                catchments.timeofday,
                catchments.catchmentid,
                catchment_stats.categorytype,		 
				CASE 
            		WHEN SUM(catchment_stats.population) = 0
            		THEN 0 
            		ELSE 10000 / SUM(catchment_stats.population) 
            		END AS ratio
         FROM catchments
		 JOIN catchment_stats
         	ON catchment_stats.catchmentid = catchments.catchmentid                
         JOIN catchmenth3map c3m 
		 	ON catchment_stats.catchmentid = c3m.catchmentid
		 GROUP BY 
		 	c3m.h3id,                
            catchments.timeofday,
            catchments.catchmentid,
            catchment_stats.categorytype
     )
	 
	 INSERT INTO step1_stats 
	 (h3id,		
			timeofday,
			categorytype,			
			catchmentid,
			ratio
	 )
	 	SELECT 
            h3id,		
			timeofday,
			categorytype,			
			catchmentid,
			ratio
		 FROM step1;

CREATE INDEX IF NOT EXISTS step1_stats_agg_index ON public.step1_stats (
    catchmentid, categorytype, timeofday
);
CREATE INDEX IF NOT EXISTS step1_stats_cid ON public.step1_stats (catchmentid);
CREATE INDEX IF NOT EXISTS step1_stats_h3id ON public.step1_stats (h3id);

"""

execute_sql(SQL)

## Step 2 calculations and population of accessibility stats

Then, step 2.

In [2]:
create_sql = """

DROP TABLE IF EXISTS accessibility_stats;

CREATE TABLE public.accessibility_stats
(
    id bigserial,
    cityid bigint,    
    categorytype text,    
    poi_category varchar(50),
    timeofday varchar(50),
    h3id char(15),
    accessibility float,
    CONSTRAINT acc_stats_id PRIMARY KEY (id)
    --- ,CONSTRAINT acc_stats_unique_key UNIQUE (cityid, categorytype, groupname, poi_category, timeofday, h3id)
);
"""

execute_sql(create_sql)

Then, populate step by step

In [3]:
stats_sql = """    

    --- Data collection for step 2. We need to:
         ---  Get all h3ids in a city and their associated population attributes
         --- For each h3id, get the catchment IDs that cover those cells
         --- Note that some cells may not have a single catchment area covering them
         --- But we still need to make sure they are captured for statistics purposes

    WITH h3_coverage as (
            SELECT DISTINCT 
                    h3demographics.h3id,
                    h3demographics.cityid,
                    h3demographics.categorytype                                        
            FROM h3demographics            
         ),
         step1 as (
            SELECT  
            step1_stats.h3id,           
            step1_stats.timeofday,
            step1_stats.categorytype,            
            pois.category as poi_category,
            SUM(step1_stats.ratio) as ratio -- if there are multiple POIs if the same type in h3, add their ratios
            FROM step1_stats 
            JOIN catchments ON catchments.catchmentid = step1_stats.catchmentid
            JOIN pois ON pois.h3id = catchments.originh3id
			GROUP BY
				step1_stats.h3id,
            	step1_stats.timeofday,
            	step1_stats.categorytype,            	
            	poi_category
         ),
         stats as (
            SELECT 
                h3_coverage.h3id,
                h3_coverage.cityid,
                h3_coverage.categorytype,                
                step1.poi_category,
                step1.timeofday,
                SUM(step1.ratio) as accessibility
            FROM h3_coverage
            LEFT JOIN step1 ON h3_coverage.h3id = step1.h3id
            WHERE 
                h3_coverage.categorytype = step1.categorytype                
            GROUP BY h3_coverage.h3id,
                h3_coverage.cityid,
                h3_coverage.categorytype,                
                step1.poi_category,
                step1.timeofday)
    INSERT
    INTO accessibility_stats (h3id, cityid, categorytype, poi_category, timeofday, accessibility)
    SELECT * FROM stats WHERE categorytype = %s AND poi_category = %s and timeofday = %s and cityid = %s
    ;
"""

In [4]:
import itertools as itt
from tqdm import tqdm

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT cityid FROM cities')
        cityids = [d[0] for d in cur.fetchall()]
        cur.execute('SELECT DISTINCT category FROM pois')
        poi_categories = [d[0] for d in cur.fetchall()]
        cur.execute('SELECT DISTINCT timeofday FROM catchments')
        timesofday = [d[0] for d in cur.fetchall()]
        cur.execute('SELECT DISTINCT categorytype FROM h3demographics')
        categorytypes = [d[0] for d in cur.fetchall()]
        combs = list(itt.product(cityids, poi_categories, timesofday, categorytypes))
        for cityid, poi_category, timeofday, categorytype in tqdm(combs):
            cur.execute(stats_sql, (
                categorytype, poi_category, timeofday, cityid
            ))
            conn.commit()


 35%|███▌      | 158/450 [42:33<3:47:15, 46.70s/it]

In [53]:
add_indexes = """
CREATE INDEX IF NOT EXISTS acc_stats_agg_index ON public.accessibility_stats (cityid, categorytype, poi_category, timeofday);
CREATE INDEX IF NOT EXISTS acc_stats_h3index ON public.accessibility_stats (h3id);
"""

execute_sql(add_indexes)

## Testing!

In [2]:
hex_id = '8944c1a8e23ffff'
import numpy as np

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:    
        #first, find all catchment areas this hex is in
        cur.execute("SELECT catchmentid FROM catchmenth3map WHERE h3id = %s", (hex_id,))
        catchments = [d[0] for d in cur.fetchall()]

        #check if that agrees with step1
        cur.execute("SELECT count(distinct catchmentid) FROM step1_stats WHERE h3id = %s", (hex_id,))
        count = cur.fetchone()[0]
        assert count == len(catchments), "The number of catchments stored in step1 table is not correct. Check!"


        #then, filter the list only catchments of Restaurants type
        cur.execute(
            """SELECT catchmentid FROM pois JOIN catchments ON pois.h3id = catchments.originh3id WHERE 
            category = 'Restaurants' and timeofday = 'morning' AND catchmentid IN %s""", 
            (tuple(catchments),)
        )
        restaurant_catchments = [d[0] for d in cur.fetchall()]        
        print("Number of unique catchments: {}".format(len(set(restaurant_catchments))))        
        print("Number of restaurants: {}".format(len(restaurant_catchments)))
        
        populations = []
        c_dict = {}
        for cid in restaurant_catchments:
            #for each catchment, get total population
            cur.execute("""SELECT SUM(population) FROM h3demographics h
            JOIN catchmenth3map c ON h.h3id = c.h3id 
            WHERE categorytype = 'Race' AND catchmentid = %s""", (cid, ))
            pop = cur.fetchone()[0]
            populations.append(pop)
            c_dict[cid] = pop

        #check if total value stored in step1 is the same (on catchment basis)
        step1s = np.array(list(c_dict.values()))
        total_step1 = (10_000 / step1s).sum()
        print("Total step1 value: {}".format(total_step1))
        cur.execute(
            """SELECT SUM(ratio) FROM step1_stats WHERE h3id = %s AND catchmentid IN %s AND categorytype = 'Race' """, 
            (hex_id, tuple(set(restaurant_catchments)))
        )
        total_step1_in_db = cur.fetchone()[0]
        print("Total step1 value in database: {}".format(total_step1_in_db))
        assert np.isclose(total_step1 - total_step1_in_db, 0), "Total step 1 value is not the same!"

        total_access = (10_000 / np.array(populations)).sum()
        print("Total accessibility in hex id: {}".format(total_access))
        cur.execute(
            """SELECT accessibility FROM accessibility_stats WHERE h3id = %s AND poi_category='Restaurants' 
            AND categorytype = 'Race' and timeofday = 'morning' """,
            (hex_id,)
        )
        total_access_in_db = cur.fetchone()[0]
        print("Total accessibility in database: {}".format(total_access_in_db))
        assert np.isclose(total_access - total_access_in_db, 0), "Total accessibility is not the same!"


AssertionError: The number of catchments stored in step1 table is not correct. Check!